In [47]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, cross
import pandas as pd
import pandas_ta as ta
from binance.client import Client
import numpy as np
import matplotlib.pyplot as plt
import math
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates
import matplotlib.pyplot as plt
client = Client()

In [48]:
def getdata(symbol,start):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'4h',start))
    frame = frame.iloc[:,:6]
    frame.columns = ['time','open','high','low','close','volume']
    frame.set_index('time',inplace=True)
    frame.index = pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    return frame
df = getdata('ETHUSDT','2021-01-01')



In [49]:
# support function
def support(df, l, n1, n2):# l is the candle (row) checked. n1 is how many candles before candle l & n2 is how many candles after candle l
    for i in range(l-n1+1,l+1):
        if(df.low[i]>df.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df.low[i] < df.low[i-1]):
            return 0
    return 1

In [50]:
def resistance(df, l, n1, n2):# l is the candle (row) checked. n1 is how many candles before candle l & n2 is how many candles after candle l
    for i in range(l-n1+1,l+1):
        if(df.low[i]<df.low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df.low[i] > df.low[i-1]):
            return 0
    return 1

In [53]:
ss,rr = [],[]
n1,n2 = 2,2
for row in range(3, 205): #len(df)):
    if support(df, row, n1, n2):
        ss.append((row, df.low[row]))
    if resistance(df,row,n1,n2):
        rr.append((row,df.high[row]))

In [54]:
s = 0
e = 200
dfpl = df[s:e]
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

c = 0
while(1):
    if(c>len(ss)-1):
        break
    fig.add_shape(type='line',x0 = ss[c][0],y0 = ss[c][1],
                    x1= e,
                    y1=ss[c][1],
                    line=dict(color='MediumPurple',width=2))
    c+=1

c = 0
while(1):
    if(c>len(rr)-1):
        break
    fig.add_shape(type='line',x0 = rr[c][0],y0 = rr[c][1],
                    x1= e,
                    y1=rr[c][1],
                    line=dict(color='RoyalBlue',width=2))
    c+=1

fig.show()